In [1]:
#!/usr/bin/env python3
import gym, gym.spaces
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [4]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)
    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [5]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
    def forward(self, x):
        return self.net(x)

In [6]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [7]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [8]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [9]:
if __name__ == "__main__":
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n
    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-frozenlake-naive")
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 0.8:
            print("Solved!")
            break
    writer.close()

0: loss=1.397, reward_mean=0.0, reward_bound=0.0
1: loss=1.383, reward_mean=0.0, reward_bound=0.0
2: loss=1.383, reward_mean=0.0, reward_bound=0.0
3: loss=1.361, reward_mean=0.0, reward_bound=0.0
4: loss=1.326, reward_mean=0.0, reward_bound=0.0
5: loss=1.357, reward_mean=0.0, reward_bound=0.0
6: loss=1.347, reward_mean=0.1, reward_bound=0.0
7: loss=1.306, reward_mean=0.0, reward_bound=0.0
8: loss=1.292, reward_mean=0.0, reward_bound=0.0
9: loss=1.301, reward_mean=0.0, reward_bound=0.0
10: loss=1.283, reward_mean=0.0, reward_bound=0.0
11: loss=1.287, reward_mean=0.0, reward_bound=0.0
12: loss=1.242, reward_mean=0.0, reward_bound=0.0
13: loss=1.174, reward_mean=0.0, reward_bound=0.0
14: loss=1.204, reward_mean=0.0, reward_bound=0.0
15: loss=1.186, reward_mean=0.0, reward_bound=0.0
16: loss=1.106, reward_mean=0.0, reward_bound=0.0
17: loss=1.089, reward_mean=0.0, reward_bound=0.0
18: loss=1.148, reward_mean=0.0, reward_bound=0.0
19: loss=1.076, reward_mean=0.0, reward_bound=0.0
20: loss=0

167: loss=0.868, reward_mean=0.1, reward_bound=0.0
168: loss=0.913, reward_mean=0.0, reward_bound=0.0
169: loss=0.952, reward_mean=0.0, reward_bound=0.0
170: loss=0.883, reward_mean=0.1, reward_bound=0.0
171: loss=0.906, reward_mean=0.0, reward_bound=0.0
172: loss=0.924, reward_mean=0.1, reward_bound=0.0
173: loss=0.866, reward_mean=0.0, reward_bound=0.0
174: loss=0.890, reward_mean=0.0, reward_bound=0.0
175: loss=0.928, reward_mean=0.0, reward_bound=0.0
176: loss=0.801, reward_mean=0.0, reward_bound=0.0
177: loss=0.787, reward_mean=0.1, reward_bound=0.0
178: loss=0.919, reward_mean=0.1, reward_bound=0.0
179: loss=0.887, reward_mean=0.0, reward_bound=0.0
180: loss=0.663, reward_mean=0.0, reward_bound=0.0
181: loss=0.743, reward_mean=0.1, reward_bound=0.0
182: loss=1.005, reward_mean=0.0, reward_bound=0.0
183: loss=0.754, reward_mean=0.0, reward_bound=0.0
184: loss=0.652, reward_mean=0.0, reward_bound=0.0
185: loss=0.731, reward_mean=0.1, reward_bound=0.0
186: loss=0.841, reward_mean=0.

336: loss=0.678, reward_mean=0.0, reward_bound=0.0
337: loss=0.704, reward_mean=0.0, reward_bound=0.0
338: loss=0.662, reward_mean=0.0, reward_bound=0.0
339: loss=0.714, reward_mean=0.0, reward_bound=0.0
340: loss=0.729, reward_mean=0.0, reward_bound=0.0
341: loss=0.677, reward_mean=0.0, reward_bound=0.0
342: loss=0.749, reward_mean=0.0, reward_bound=0.0
343: loss=0.760, reward_mean=0.0, reward_bound=0.0
344: loss=0.696, reward_mean=0.0, reward_bound=0.0
345: loss=0.752, reward_mean=0.0, reward_bound=0.0
346: loss=0.756, reward_mean=0.0, reward_bound=0.0
347: loss=0.624, reward_mean=0.0, reward_bound=0.0
348: loss=0.711, reward_mean=0.0, reward_bound=0.0
349: loss=0.807, reward_mean=0.0, reward_bound=0.0
350: loss=0.738, reward_mean=0.0, reward_bound=0.0
351: loss=0.763, reward_mean=0.0, reward_bound=0.0
352: loss=0.736, reward_mean=0.0, reward_bound=0.0
353: loss=0.618, reward_mean=0.0, reward_bound=0.0
354: loss=0.628, reward_mean=0.0, reward_bound=0.0
355: loss=0.811, reward_mean=0.

503: loss=0.367, reward_mean=0.0, reward_bound=0.0
504: loss=0.392, reward_mean=0.0, reward_bound=0.0
505: loss=0.489, reward_mean=0.1, reward_bound=0.0
506: loss=0.454, reward_mean=0.0, reward_bound=0.0
507: loss=0.504, reward_mean=0.0, reward_bound=0.0
508: loss=0.501, reward_mean=0.0, reward_bound=0.0
509: loss=0.465, reward_mean=0.0, reward_bound=0.0
510: loss=0.494, reward_mean=0.0, reward_bound=0.0
511: loss=0.542, reward_mean=0.0, reward_bound=0.0
512: loss=0.480, reward_mean=0.0, reward_bound=0.0
513: loss=0.466, reward_mean=0.0, reward_bound=0.0
514: loss=0.485, reward_mean=0.0, reward_bound=0.0
515: loss=0.479, reward_mean=0.0, reward_bound=0.0
516: loss=0.430, reward_mean=0.0, reward_bound=0.0
517: loss=0.464, reward_mean=0.0, reward_bound=0.0
518: loss=0.422, reward_mean=0.0, reward_bound=0.0
519: loss=0.562, reward_mean=0.0, reward_bound=0.0
520: loss=0.402, reward_mean=0.0, reward_bound=0.0
521: loss=0.427, reward_mean=0.0, reward_bound=0.0
522: loss=0.524, reward_mean=0.

671: loss=0.007, reward_mean=0.0, reward_bound=0.0
672: loss=0.007, reward_mean=0.1, reward_bound=0.0
673: loss=0.068, reward_mean=0.0, reward_bound=0.0
674: loss=0.050, reward_mean=0.0, reward_bound=0.0
675: loss=0.141, reward_mean=0.0, reward_bound=0.0
676: loss=0.005, reward_mean=0.0, reward_bound=0.0
677: loss=0.070, reward_mean=0.1, reward_bound=0.0
678: loss=0.007, reward_mean=0.1, reward_bound=0.0
679: loss=0.005, reward_mean=0.0, reward_bound=0.0
680: loss=0.006, reward_mean=0.1, reward_bound=0.0
681: loss=0.086, reward_mean=0.1, reward_bound=0.0
682: loss=0.008, reward_mean=0.1, reward_bound=0.0
683: loss=0.006, reward_mean=0.1, reward_bound=0.0
684: loss=0.076, reward_mean=0.0, reward_bound=0.0
685: loss=0.006, reward_mean=0.0, reward_bound=0.0
686: loss=0.053, reward_mean=0.1, reward_bound=0.0
687: loss=0.125, reward_mean=0.0, reward_bound=0.0
688: loss=0.006, reward_mean=0.0, reward_bound=0.0
689: loss=0.005, reward_mean=0.0, reward_bound=0.0
690: loss=0.004, reward_mean=0.

...

4950: loss=0.000, reward_mean=0.1, reward_bound=0.0
4951: loss=0.000, reward_mean=0.0, reward_bound=0.0
4952: loss=0.000, reward_mean=0.0, reward_bound=0.0
4953: loss=0.000, reward_mean=0.0, reward_bound=0.0
4954: loss=0.000, reward_mean=0.0, reward_bound=0.0
4955: loss=0.000, reward_mean=0.0, reward_bound=0.0
4956: loss=0.000, reward_mean=0.0, reward_bound=0.0
4957: loss=0.000, reward_mean=0.1, reward_bound=0.0
4958: loss=0.000, reward_mean=0.0, reward_bound=0.0
4959: loss=0.000, reward_mean=0.1, reward_bound=0.0
4960: loss=0.000, reward_mean=0.0, reward_bound=0.0
4961: loss=0.000, reward_mean=0.1, reward_bound=0.0
4962: loss=0.000, reward_mean=0.1, reward_bound=0.0
4963: loss=0.000, reward_mean=0.0, reward_bound=0.0
4964: loss=0.000, reward_mean=0.1, reward_bound=0.0
4965: loss=0.000, reward_mean=0.0, reward_bound=0.0
4966: loss=0.000, reward_mean=0.0, reward_bound=0.0
4967: loss=0.000, reward_mean=0.0, reward_bound=0.0
4968: loss=0.000, reward_mean=0.1, reward_bound=0.0
4969: loss=0

5108: loss=0.000, reward_mean=0.0, reward_bound=0.0
5109: loss=0.000, reward_mean=0.0, reward_bound=0.0
5110: loss=0.000, reward_mean=0.0, reward_bound=0.0
5111: loss=0.000, reward_mean=0.1, reward_bound=0.0
5112: loss=0.000, reward_mean=0.0, reward_bound=0.0
5113: loss=0.000, reward_mean=0.0, reward_bound=0.0
5114: loss=0.000, reward_mean=0.0, reward_bound=0.0
5115: loss=0.000, reward_mean=0.0, reward_bound=0.0
5116: loss=0.000, reward_mean=0.0, reward_bound=0.0
5117: loss=0.000, reward_mean=0.1, reward_bound=0.0
5118: loss=0.000, reward_mean=0.1, reward_bound=0.0
5119: loss=0.000, reward_mean=0.1, reward_bound=0.0
5120: loss=0.000, reward_mean=0.1, reward_bound=0.0
5121: loss=0.000, reward_mean=0.0, reward_bound=0.0
5122: loss=0.000, reward_mean=0.0, reward_bound=0.0
5123: loss=0.000, reward_mean=0.0, reward_bound=0.0
5124: loss=0.000, reward_mean=0.0, reward_bound=0.0
5125: loss=0.000, reward_mean=0.1, reward_bound=0.0
5126: loss=0.000, reward_mean=0.0, reward_bound=0.0
5127: loss=0

5266: loss=0.000, reward_mean=0.1, reward_bound=0.0
5267: loss=0.000, reward_mean=0.0, reward_bound=0.0
5268: loss=0.000, reward_mean=0.0, reward_bound=0.0
5269: loss=0.000, reward_mean=0.1, reward_bound=0.0
5270: loss=0.000, reward_mean=0.0, reward_bound=0.0
5271: loss=0.000, reward_mean=0.0, reward_bound=0.0
5272: loss=0.000, reward_mean=0.0, reward_bound=0.0
5273: loss=0.000, reward_mean=0.1, reward_bound=0.0
5274: loss=0.000, reward_mean=0.1, reward_bound=0.0
5275: loss=0.000, reward_mean=0.0, reward_bound=0.0
5276: loss=0.000, reward_mean=0.1, reward_bound=0.0
5277: loss=0.000, reward_mean=0.0, reward_bound=0.0
5278: loss=0.000, reward_mean=0.1, reward_bound=0.0
5279: loss=0.000, reward_mean=0.0, reward_bound=0.0
5280: loss=0.000, reward_mean=0.0, reward_bound=0.0
5281: loss=0.000, reward_mean=0.0, reward_bound=0.0
5282: loss=0.000, reward_mean=0.0, reward_bound=0.0
5283: loss=0.000, reward_mean=0.1, reward_bound=0.0
5284: loss=0.000, reward_mean=0.1, reward_bound=0.0
5285: loss=0

5428: loss=0.000, reward_mean=0.1, reward_bound=0.0
5429: loss=0.000, reward_mean=0.0, reward_bound=0.0
5430: loss=0.000, reward_mean=0.1, reward_bound=0.0
5431: loss=0.000, reward_mean=0.0, reward_bound=0.0
5432: loss=0.000, reward_mean=0.1, reward_bound=0.0
5433: loss=0.000, reward_mean=0.0, reward_bound=0.0
5434: loss=0.000, reward_mean=0.1, reward_bound=0.0
5435: loss=0.000, reward_mean=0.0, reward_bound=0.0
5436: loss=0.000, reward_mean=0.0, reward_bound=0.0
5437: loss=0.000, reward_mean=0.0, reward_bound=0.0
5438: loss=0.000, reward_mean=0.1, reward_bound=0.0
5439: loss=0.000, reward_mean=0.1, reward_bound=0.0
5440: loss=0.000, reward_mean=0.0, reward_bound=0.0
5441: loss=0.000, reward_mean=0.1, reward_bound=0.0
5442: loss=0.000, reward_mean=0.0, reward_bound=0.0
5443: loss=0.000, reward_mean=0.1, reward_bound=0.0
5444: loss=0.000, reward_mean=0.1, reward_bound=0.0
5445: loss=0.000, reward_mean=0.1, reward_bound=0.0
5446: loss=0.000, reward_mean=0.0, reward_bound=0.0
5447: loss=0

KeyboardInterrupt: 